### Gör scraping av en eller flera omgångar och returnerar en DataFrame
- Körs med v75_scraping(resultat=False,history=True)
- Parametrar: resultat=True/False, history=True/False
- Input: omg_att_spela_link.csv() med en eler flera omgångar/veckor

In [1]:
# %%
# from IPython import get_ipython


In [2]:
import pandas as pd
import numpy as np
import os

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys

import streamlit as st
import fixa_mer_features as ff2
import time

# %% [markdown]
# # Load all the links with races

# %%
# import logging
# logging.basicConfig(filename='app.log', filemode='w',
#                     format='%(name)s - %(message)s', level=logging.INFO)

# %% [markdown]


## Functions to get all the data

In [3]:
from selenium.webdriver.chrome.options import Options


def get_webdriver(headless=True):
    from selenium import webdriver


    # instance of Options class allows
    # us to configure Headless Chrome
    options = Options()

    if headless:
        # this parameter tells Chrome that
        # it should be run without UI (Headless)
        options.headless = True

    # initializing webdriver for Chrome with our options
    driver = webdriver.Chrome(
        executable_path='C:\\Users\peter\\Documents\\MyProjects\\gecko\\chromedriver.exe',options = options)

    # # getting GeekForGeeks webpage
    # driver.get('https://www.geeksforgeeks.org')

    # We can also get some information
    # about page in browser.
    # So let's output webpage title into
    # terminal to be sure that the browser
    # is actually running.
    print(driver.title)

    return driver

get_webdriver()

C:\Users\peter\AppData\Local\Temp/ipykernel_39848/3686862442.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(


<selenium.webdriver.chrome.webdriver.WebDriver (session="0d6ba9b3aadf72fa3a5c21167af0c3c1")>

In [4]:

def städa_och_rensa(df, history):
    före = len(df)
    df = ff2.fixa_mer_features(df, history)
    efter = len(df)
    if före-efter != 0:
        print('rensade totalt bort', före-efter,
              'hästar i städa_och_rensa. Från', före, 'till', efter)
    return df


def scrape_utdelning(the_driver):
    # gammal lösn: utdelningar = the_driver.find_elements_by_class_name("css-mxas0-Payouts-styles--amount")

    utdelningar = the_driver.find_elements(
        By.CLASS_NAME, "css-fu45i8-Payouts-styles--amount")

    utd7 = utdelningar[0].text.replace(' ', '')
    utd7 = utd7.replace('kr', '')
    utd7 = utd7.replace('Jackpot', '0')

    utd6 = utdelningar[1].text.replace(' ', '')
    utd6 = utd6.replace('kr', '')
    utd6 = utd6.replace('Jackpot', '0')

    utd5 = utdelningar[2].text.replace(' ', '')
    utd5 = utd5.replace('kr', '')
    utd5 = utd5.replace('Jackpot', '0')

    return int(utd7), int(utd6), int(utd5)


In [5]:

# hämta utdelnings-info och spara på fil

def utdelning(driver_r, dat, bana):
    utd7, utd6, utd5 = scrape_utdelning(driver_r)
    print(f'utdelning: {utd7}, {utd6}, {utd5}')
    utd_file = 'C:\\Users\\peter\\Documents\\MyProjects\\PyProj\\Trav\\spel\\utdelning.csv'
    utdelning = pd.read_csv(utd_file)
    utd = pd.DataFrame([[dat, bana, utd7, utd6, utd5]])
    utd.columns = utdelning.columns
    utdelning = pd.concat([utdelning, utd])
    utdelning.drop_duplicates(['datum'], inplace=True)

    utdelning.sort_values(by=['datum'], inplace=True)
    utdelning.reset_index(drop=True, inplace=True)
    utdelning.to_csv(utd_file, index=False)


## resultat i startnummer-ordning

In [6]:
def inkludera_resultat(res_avd, anr):
    res_rader = res_avd.find_elements_by_class_name(
        "startlist__row")  # alla rader i loppet
    res_startnr = res_avd.find_elements_by_class_name(
        "css-1jc4209-horseview-styles--startNumber")  # alla startnr
    d = {'plac': [], 'startnr': []}
    for nr, rad in enumerate(res_rader):
        plac = rad.text.split(' ')[0]

        d['plac'].append(plac)
        d['startnr'].append(res_startnr[nr].text)

    temp_df = pd.DataFrame(d)
    temp_df['startnr'] = temp_df.startnr.astype(int)
    temp_df.sort_values(by='startnr', ascending=True, inplace=True)

    return temp_df.plac.to_list()


## Do actual scraping

In [7]:
def do_scraping(driver_s, driver_r, history, datum):  # get data from web site
    # logging.basicConfig(filename='app.log', filemode='w',
    #                     format='%(name)s - %(message)s', level=logging.INFO)

    vdict = {'datum': [], 'bana': [], 'avd': [], 'startnr': [], 'häst': [], 'ålder': [], 'kön': [], 'kusk': [], 'lopp_dist': [],
             'start': [], 'dist': [], 'pris': [], 'spår': [], 'streck': [], 'vodds': [], 'podds': [], 'kr': [], }
    if driver_r:
        vdict.update({'plac': []})
    if history:
        vdict.update(
            {'h1_dat': [], 'h1_bana': [], 'h1_kusk': [], 'h1_plac': [], 'h1_dist': [], 'h1_spår': [], 'h1_odds': [], 'h1_pris': [], 'h1_kmtid': [],
             'h2_dat': [], 'h2_bana': [], 'h2_kusk': [], 'h2_plac': [], 'h2_dist': [], 'h2_spår': [], 'h2_odds': [], 'h2_pris': [], 'h2_kmtid': [],
             'h3_dat': [], 'h3_bana': [], 'h3_kusk': [], 'h3_plac': [], 'h3_dist': [], 'h3_spår': [], 'h3_odds': [], 'h3_pris': [], 'h3_kmtid': [],
             'h4_dat': [], 'h4_bana': [], 'h4_kusk': [], 'h4_plac': [], 'h4_dist': [], 'h4_spår': [], 'h4_odds': [], 'h4_pris': [], 'h4_kmtid': [],
             'h5_dat': [], 'h5_bana': [], 'h5_kusk': [], 'h5_plac': [], 'h5_dist': [], 'h5_spår': [], 'h5_odds': [], 'h5_pris': [], 'h5_kmtid': [],
             })

    start_time = time.perf_counter()

    if driver_r:
        result_tab = driver_r.find_elements_by_class_name(
            'game-table')[:]  # alla lopp med resultatordning
        if len(result_tab) == 8:
            result_tab = result_tab[1:]
        print('ant resultat', len(result_tab))

    start_tab = driver_s.find_elements_by_class_name(
        'game-table')[:]  # alla lopp med startlistor
    if len(start_tab) == 8:
        start_tab = start_tab[1:]
    print('ant lopp', len(start_tab))

    comb = driver_s.find_elements_by_class_name(
        'race-combined-info')  # alla bana,dist,start
    priselement = driver_s.find_elements_by_class_name(
        'css-1lnkuf6-startlistraceinfodetails-styles--infoContainer')
    NOK, EUR = False, False
    for p in priselement:
        if 'EUR' in p.text:
            EUR = True
            break
        elif 'NOK' in p.text:
            NOK = True
            break
    print('EUR:', EUR, 'NOK:', NOK)
    # alla lopps priser
    priser = [p.text for p in priselement if 'Pris:' in p.text]

    if len(priser) == 0:
        priser = [p.text for p in priselement if 'åriga' not in p.text and (
            'EUR' in p.text or 'NOK' in p.text)]  # alla lopps priser
    print('priser', priser)
    print('Ant priser', len(priser))
    # ett lopp (de häst-relaterade som inte kan bli 'missing' tar jag direkt på loppnivå)
    for anr, avd in enumerate(start_tab):
        # logging.warning(datum+' avd: '+str(avd))

        bana = comb[anr].text.split('\n')[0]
        lopp_dist = comb[anr].text.split('\n')[1].split(' ')[0][:-1]
        start = comb[anr].text.split('\n')[1].split(' ')[1]

        if EUR:
            pris = priser[anr].split('-')[0] + '0'    # Euro -> SEK
        elif NOK:
            pris = priser[anr].split('-')[0].replace('.', '')  # NOK -> SEK
        else:
            pris = priser[anr].split('-')[0].split(' ')[1]

        print('pris:', pris)
        names = avd.find_elements_by_class_name(
            "horse-name")  # alla hästar/kön/åldet i loppet
        voddss = avd.find_elements_by_class_name(
            "vOdds-col")[1:]  # vodds i loppet utan rubrik
        poddss = avd.find_elements_by_class_name(
            "pOdds-col")[1:]  # podds i loppet utan rubrik
        rader = avd.find_elements_by_class_name(
            "startlist__row")  # alla rader i loppet
        strecks = avd.find_elements_by_class_name(
            "betDistribution-col")[1:]  # streck i loppet  utan rubrik
        print('ant names,vodds,podds,rader,streck', len(
            names), len(voddss), len(poddss), len(strecks))
        if driver_r:
            # resultat från resultatsidan
            res_avd = result_tab[anr]
            # placeringar sorterade efter startnr för en avd
            places = inkludera_resultat(res_avd, anr+1)
            # res_startnr = res_avd.find_elements_by_class_name("css-1jc4209-horseview-styles--startNumber")[1:]
            vdict['plac'] += places    # konkatenera listorna

        print('AVD', anr+1, bana, lopp_dist, start, end=' ')

        for r, rad in enumerate(rader):
            # en häst
            # logging.warning(r)

            vdict['datum'].append(datum)
            vdict['bana'].append(bana)
            vdict['avd'].append(anr+1)
            vdict['startnr'].append(r+1)
            vdict['häst'].append(names[r].text)
            vdict['start'].append(start)
            vdict['lopp_dist'].append(lopp_dist)
            vdict['pris'].append(pris)
            vdict['vodds'].append(voddss[r].text)  # vodds i lopp 1 utan rubrik
            vdict['podds'].append(poddss[r].text)  # podds i lopp 1 utan rubrik
            # streck i lopp 1  utan rubrik
            vdict['streck'].append(strecks[r].text)
            # kusk måste tas på hästnivå (pga att driver-col finns också i hist)
            vdict['kusk'].append(
                rad.find_elements_by_class_name("driver-col")[0].text)
            vdict['ålder'].append(
                rad.find_elements_by_class_name("horse-age")[0].text)
            vdict['kön'].append(
                rad.find_elements_by_class_name("horse-sex")[0].text)
            vdict['kr'].append(rad.find_elements_by_class_name(
                "earningsPerStart-col")[0].text)  # kr/start i lopp 1 utan rubrik

            dist_sp = rad.find_elements_by_class_name(
                "postPositionAndDistance-col")  # dist och spår i lopp 1 utan rubrik
            vdict['dist'].append(dist_sp[0].text.split(':')[0])
            vdict['spår'].append(dist_sp[0].text.split(':')[1])

            if history:
                ## history från startlistan ##
                hist = avd.find_elements_by_class_name(
                    "start-info-panel")  # all history för loppet

                h_dates = hist[r].find_elements_by_class_name('date-col')[1:]
                h_kuskar = hist[r].find_elements_by_class_name(
                    'driver-col')[1:]
                h_banor = hist[r].find_elements_by_class_name('track-col')[1:]
                h_plac = hist[r].find_elements_by_class_name(
                    'place-col')[1:]  # obs varannan rad (10 rader)
                h_plac = h_plac[::2]   # ta ut varannat värde

                h_dist = hist[r].find_elements_by_class_name(
                    'distance-col')[1:]
                h_spår = hist[r].find_elements_by_class_name(
                    'position-col')[1:]
                h_kmtid = hist[r].find_elements_by_class_name('kmTime-col')[1:]
                h_odds = hist[r].find_elements_by_class_name('odds-col')[1:]
                h_pris = hist[r].find_elements_by_class_name(
                    'firstPrize-col')[1:]

                for h, d in enumerate(h_dates):
                    fld = 'h'+str(h+1)+'_'
                    dtext = d.text
                    vdict[fld+'dat'].append(dtext)
                    vdict[fld+'bana'].append(h_banor[h].text)
                    vdict[fld+'kusk'].append(h_kuskar[h].text)
                    vdict[fld+'plac'].append(h_plac[h].text)
                    vdict[fld+'dist'].append(h_dist[h].text)
                    vdict[fld+'spår'].append(h_spår[h].text)
                    vdict[fld+'kmtid'].append(h_kmtid[h].text)
                    vdict[fld+'odds'].append(h_odds[h].text)
                    vdict[fld+'pris'].append(h_pris[h].text)
                if len(h_dates) < 5:
                    # Duplicera hist_4
                    fld = 'h'+str(5)+'_'
                    h = 3
                    vdict[fld+'dat'].append(dtext)
                    vdict[fld+'bana'].append(h_banor[h].text)
                    vdict[fld+'kusk'].append(h_kuskar[h].text)
                    vdict[fld+'plac'].append(h_plac[h].text)
                    vdict[fld+'dist'].append(h_dist[h].text)
                    vdict[fld+'spår'].append(h_spår[h].text)
                    vdict[fld+'kmtid'].append(h_kmtid[h].text)
                    vdict[fld+'odds'].append(h_odds[h].text)
                    vdict[fld+'pris'].append(h_pris[h].text)

            print('.', end='')
        print()

    print('\ndet tog', round(time.perf_counter() - start_time, 3), 'sekunder')

    return vdict, bana


## Anpassa startlistan innan scrape

In [8]:

def anpassa(driver_s):
    sl = driver_s.find_elements(By.CLASS_NAME,"startlist")
    # print('sl all text\n', sl[0].text)
    # print('sl end')
    buts = sl[0].find_elements(By.CLASS_NAME,"css-eugx3a-startlistoptionsview-styles--configButton-Button--buttonComponent")

    buts[0].click()
    time.sleep(1)
    print('klickade på', buts[0].text)
   
    anpassa2(driver_s)



In [9]:
# sl = driver_s.find_elements_by_class_name("startlist")
# print('sl all text\n', sl[0].text)
# print('sl end')
# buts = sl[0].find_elements_by_class_name("css-eugx3a-startlistoptionsview-styles--configButton-Button--buttonComponent")
# print('buts1\n', buts[1].text)
# buts[2].click()



In [10]:
def anpassa2(driver_s):
    tics = driver_s.find_elements_by_class_name("css-1hngy38-Checkbox-styles--label")
    driver_s.implicitly_wait(5)     # seconds
    # print('len tics',len(tics))

    flag1 = flag2 = flag3 = flag4 = flag5 = flag6 = flag7 = flag8 = flag9 = True
    for t in tics:
        if not (flag1 or flag2 or flag3 or flag4 or flag5 or flag6 or flag7 or flag8 or flag9):
            print('anpassa klar - break')
            break
        if t.text == '':
            continue

        if flag1 and t.text == 'VÄRMNINGS­VIDEO':
            t.click()
            # print('värmn')
            flag1 = False
        elif flag2 and t.text == 'UTÖKA ALLA STARTLISTOR SAMTIDIGT':
            t.click()
            # print('utöka')
            flag2 = False
        elif flag3 and t.text == 'TIPSKOMMENTARER':
            t.click()
            # print('tips')
            flag3 = False
        elif flag4 and ('LOPPKOMMENTARER' in t.text):
            t.click()
            # print('komment')
            flag4 = False
        elif flag5 and t.text == 'KR/START':
            t.click()
            # print('kr')
            flag5 = False
        elif flag6 and t.text == 'DISTANS OCH SPÅR':
            t.click()
            # print('dist')
            flag6 = False
        elif flag7 and t.text == 'V-ODDS':
            # t.click()
            print('hoppar över voods click (verkar vara förifyllt')
            flag7 = False
        elif flag8 and t.text == 'P-ODDS':
            t.click()
            # print('podds')
            flag8 = False
        elif flag9 and t.text == 'HÄSTENS KÖN & ÅLDER':
            t.click()
            # print('kön')
            flag9 = False

    ## Spara ##
    driver_s.implicitly_wait(5)     # seconds
    driver_s.find_elements_by_class_name(
        "css-1fh4n7y-Button-styles--root-PrimaryButton-styles--root-StartlistDisplayOptionsDialog-styles--saveButton-PrimaryButton--PrimaryButton-StartlistDisplayOptionsDialog-styles--saveButton")[0].click()
# anpassa(driver_s)


## Starta Här!
### Öppnar v75 sidorna (startlist ev resultat) och samlar all info per avd/häst


In [11]:

def v75_scraping(resultat=False, history=False, driver_s=None, driver_r=None):
    ### Hela loopen med alla lopp i alla veckor i omg_df ###
    omg_df = pd.read_csv(
        'C:\\Users\\peter\\Documents\\MyProjects\\PyProj\\Trav\\spel\\omg_att_spela_link.csv')
    df = pd.DataFrame()

    if not driver_r:
        if resultat:
            driver_r = get_webdriver()  # get web driver for results
        else:
            driver_r = None
    
    if not driver_s:
        driver_s = get_webdriver()  # web driver för startlista
    
    for enum, omg in enumerate(omg_df.Link.values):
        print(f'omgång {enum+1}:', omg)
        datum = omg.split('spel/')[1][0:10]

        driver_s.get(omg)              # öppna startlista
        driver_s.implicitly_wait(5)     # seconds

        if enum == 0:  # första gången startlista
            # ok till första popup om kakor
            driver_s.find_element_by_id("onetrust-accept-btn-handler").click()
            driver_s.fullscreen_window()
            driver_s.implicitly_wait(5)     # seconds

            driver_s.find_elements_by_class_name(
                "race-info-toggle")[1].click()  # prissummor mm
            driver_s.implicitly_wait(5)     # seconds

            anpassa(driver_s)

        # resultat
        if resultat:
            driver_r.get(omg+'/resultat')   # öppna resultat
            driver_r.implicitly_wait(5)     # seconds
            if enum == 0:
                # ok till första popup om kakor
                driver_r.find_element_by_id(
                    "onetrust-accept-btn-handler").click()
                driver_r.fullscreen_window()

        # scraping
        komplett, bana = do_scraping(driver_s, driver_r, history, datum)
        temp_df = pd.DataFrame(komplett)
        df = pd.concat([df, temp_df])

        # utdelning
        if resultat:
            utdelning(driver_r, datum, bana)  # utdelning för denna omgång
    
    #######
    driver_s.quit()
    if resultat:
        driver_r.quit()

    strukna = df.loc[df.vodds == 'EJ', :].copy()  # Ta bort all strukna

    df = städa_och_rensa(df, history)

    return df, strukna

    # df.to_csv('lopp_to_learn.csv',index=False,date_format='%s') # spara


In [12]:
driver_s = get_webdriver(headless=True)  # web driver för startlista                 
driver_r = get_webdriver(headless=True)  # web driver för resultat
df, strukna = v75_scraping(resultat=True, history=True, driver_s=driver_s, driver_r=driver_r)
df

C:\Users\peter\AppData\Local\Temp/ipykernel_39848/3686862442.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(




omgång 1: https://www.atg.se/spel/2022-05-07/V75/


C:\Users\peter\AppData\Local\Temp/ipykernel_39848/1855990061.py:25: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver_s.find_element_by_id("onetrust-accept-btn-handler").click()
C:\Users\peter\AppData\Local\Temp/ipykernel_39848/1855990061.py:29: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver_s.find_elements_by_class_name(


klickade på Anpassa


C:\Users\peter\AppData\Local\Temp/ipykernel_39848/3687056577.py:2: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  tics = driver_s.find_elements_by_class_name("css-1hngy38-Checkbox-styles--label")


...

C:\Users\peter\AppData\Local\Temp/ipykernel_39848/3687056577.py:53: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver_s.find_elements_by_class_name(
C:\Users\peter\AppData\Local\Temp/ipykernel_39848/1855990061.py:41: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver_r.find_element_by_id(
C:\Users\peter\AppData\Local\Temp/ipykernel_39848/1955160036.py:21: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  result_tab = driver_r.find_elements_by_class_name(
C:\Users\peter\AppData\Local\Temp/ipykernel_39848/1955160036.py:27: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  start_tab = driver_s.find_elements_by_class_name(
C:\Users\peter\AppData\Local\Temp/ipykernel_39848/1955160036.py:33: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements()

ant resultat 7
ant lopp 7
EUR: False NOK: False
priser ['Pris: 1.000.000-500.000-250.000-120.000-80.000-50.000 kr (6 priser). Lägst 1.500 kr till alla tävlande.', 'Pris: 110.000-55.000-32.000-19.000-12.500-10.000-6.500-5.000 kr (8 priser). Lägst 2.500 kr till alla tävlande.', 'Pris: 1.000.000-500.000-250.000-120.000-80.000-50.000 kr (6 priser). Lägst 1.500 kr till alla tävlande.', 'Pris: 250.000-125.000-70.000-39.000-24.000-16.000-10.000-6.000 kr (8 priser). Lägst 1.500 kr till alla tävlande.', 'Pris: 110.000-55.000-32.000-19.000-12.500-10.000-6.500-5.000 kr (8 priser). Lägst 2.500 kr till alla tävlande.', 'Pris: 125.000-62.500-34.000-21.000-13.500-10.500-7.000-5.000 kr (8 priser). Lägst 2.500 kr till alla tävlande.', 'Pris: 1.500.000-700.000-325.000-150.000-75.000-40.000-30.000-25.000 kr (8 priser). Lägst 2.500 kr till alla tävlande.']
Ant priser 7
pris: 1.000.000
ant names,vodds,podds,rader,streck 12 12 12 12


c:\Users\peter\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:464: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")


AVD 1 ÅBY 2140 AUTOSTART 

IndexError: list index out of range

In [ ]:
driver_s.quit()
driver_r.quit()

In [ ]:
# concat matchar kolumner oavsett ordning!
dfall = pd.read_csv('all_data.csv')
assert dfall.shape[1] == df.shape[1], f'antal kolumner {dfall.shape[1]} resp {df.shape[1]} skall vara lika'
assert set(dfall.columns) == set(df.columns), ' det skall vara samma kolumnnamn'
dfconc = pd.concat([dfall, df])
assert dfconc.shape[0] == dfall.shape[0] + df.shape[0], 'antal rader skall vara summan'
assert dfconc.shape[1] == dfall.shape[1], 'antal kolumner skall vara oförändrat'
df = df[dfall.columns]
df

In [3]:
dfall = pd.read_csv('all_data.csv')
dfall.datum.unique()

array(['2014-12-28', '2015-01-03', '2015-01-10', '2015-01-17',
       '2015-01-24', '2015-01-25', '2015-01-31', '2015-02-07',
       '2015-02-14', '2015-02-21', '2015-02-28', '2015-03-01',
       '2015-03-07', '2015-03-14', '2015-03-21', '2015-03-28',
       '2015-04-03', '2015-04-05', '2015-04-11', '2015-04-18',
       '2015-04-25', '2015-05-02', '2015-05-09', '2015-05-10',
       '2015-05-16', '2015-05-17', '2015-05-23', '2015-05-30',
       '2015-05-31', '2015-06-06', '2015-06-13', '2015-06-14',
       '2015-06-21', '2015-06-27', '2015-07-02', '2015-07-04',
       '2015-07-11', '2015-07-18', '2015-07-19', '2015-07-25',
       '2015-07-28', '2015-08-01', '2015-08-08', '2015-08-15',
       '2015-08-19', '2015-08-22', '2015-08-25', '2015-08-29',
       '2015-09-05', '2015-09-06', '2015-09-12', '2015-09-19',
       '2015-09-26', '2015-09-27', '2015-10-03', '2015-10-10',
       '2015-10-17', '2015-10-24', '2015-10-25', '2015-10-31',
       '2015-11-07', '2015-11-08', '2015-11-14', '2015-